# Tính Toán Giờ Công Nhân Viên

Notebook này sẽ tính toán số giờ công cho từng nhân viên dựa trên dữ liệu timesheet từ file Excel với các quy tắc:
- Nếu vào làm trước 7h30 thì tính là 7h30
- Nghỉ trưa 1 tiếng
- Nhóm dữ liệu theo tên nhân viên và ngày

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Thư viện đã được import thành công!")

Thư viện đã được import thành công!


## 2. Load Excel Data

In [13]:
# Đọc file Excel
file_path = 'data/file1.xlsx'

try:
    # Đọc dữ liệu từ file Excel
    df = pd.read_excel(file_path)
    print(f"Đã đọc thành công file {file_path}")
    print(f"Số dòng: {len(df)}")
    print(f"Số cột: {len(df.columns)}")
    print("\nTên các cột:")
    print(df.columns.tolist())
    print("\nMẫu dữ liệu:")
    print(df.head())
except FileNotFoundError:
    print(f"Không tìm thấy file {file_path}")
    print("Hãy đảm bảo file tồn tại trong thư mục data/")
except Exception as e:
    print(f"Lỗi khi đọc file: {e}")

Đã đọc thành công file data/file1.xlsx
Số dòng: 212
Số cột: 6

Tên các cột:
['Mã', 'Họ tên', 'Vị trí', 'Phòng ban', 'Ngày chốt', 'Giờ chốt']

Mẫu dữ liệu:
         Mã               Họ tên                  Vị trí  \
0  25012101  TRẦN THỊ NGỌC DUYÊN  Nhân viên Nhà Đóng gói   
1  25012101  TRẦN THỊ NGỌC DUYÊN  Nhân viên Nhà Đóng gói   
2  25012101  TRẦN THỊ NGỌC DUYÊN  Nhân viên Nhà Đóng gói   
3  25012101  TRẦN THỊ NGỌC DUYÊN  Nhân viên Nhà Đóng gói   
4  25012101  TRẦN THỊ NGỌC DUYÊN  Nhân viên Nhà Đóng gói   

                                           Phòng ban   Ngày chốt  Giờ chốt  
0  Công Ty TNHH MTV The Fruit Republic Cần Thơ/Nh...  2025-05-21  07:30:00  
1  Công Ty TNHH MTV The Fruit Republic Cần Thơ/Nh...  2025-05-21  18:05:00  
2  Công Ty TNHH MTV The Fruit Republic Cần Thơ/Nh...  2025-05-22  07:30:00  
3  Công Ty TNHH MTV The Fruit Republic Cần Thơ/Nh...  2025-05-22  16:39:00  
4  Công Ty TNHH MTV The Fruit Republic Cần Thơ/Nh...  2025-05-23  07:30:00  


## 3. Data Preprocessing and Validation

Tiền xử lý dữ liệu và đảm bảo định dạng thời gian chính xác

In [14]:
# Hàm tiền xử lý dữ liệu cho cấu trúc mới
def preprocess_timesheet_data_v2(df):
    """
    Tiền xử lý dữ liệu timesheet với cấu trúc Ngày chốt và Giờ chốt
    """
    # Tạo bản sao để không thay đổi dữ liệu gốc
    df_processed = df.copy()
    
    # Chuẩn hóa tên cột
    df_processed = df_processed.rename(columns={
        'Họ tên': 'employee_name',
        'Ngày chốt': 'date',
        'Giờ chốt': 'time_stamp'
    })
    
    # Chuyển đổi cột ngày
    df_processed['date'] = pd.to_datetime(df_processed['date'])
    
    # Chuyển đổi cột giờ
    df_processed['time_stamp'] = pd.to_datetime(df_processed['time_stamp'], format='%H:%M:%S').dt.time
    
    # Sắp xếp theo nhân viên, ngày và giờ
    df_processed = df_processed.sort_values(['employee_name', 'date', 'time_stamp'])
    
    return df_processed

def calculate_work_hours_v2(df):
    """
    Tính toán giờ làm việc và giờ OT từ dữ liệu chấm công
    """
    result_data = []
    
    # Group theo nhân viên và ngày
    for (employee, date), group in df.groupby(['employee_name', 'date']):
        # Sắp xếp theo thời gian trong ngày
        group = group.sort_values('time_stamp')
        times = group['time_stamp'].tolist()
        
        if len(times) >= 2:
            # Lấy thời gian đầu tiên là check_in, cuối cùng là check_out
            check_in = times[0]
            check_out = times[-1]
            
            # Áp dụng quy tắc: nếu vào trước 7:30 thì tính là 7:30
            earliest_start = time(7, 30)
            if check_in < earliest_start:
                adjusted_check_in = earliest_start
            else:
                adjusted_check_in = check_in
            
            # Tính tổng thời gian
            check_in_dt = datetime.combine(datetime.today(), adjusted_check_in)
            check_out_dt = datetime.combine(datetime.today(), check_out)
            
            # Nếu check_out < check_in (qua ngày), thêm 1 ngày
            if check_out_dt < check_in_dt:
                check_out_dt += timedelta(days=1)
            
            total_time = check_out_dt - check_in_dt
            total_hours = total_time.total_seconds() / 3600
            
            # Trừ 1 tiếng nghỉ trưa
            work_time = max(0, total_hours - 1)
            
            # Tính giờ thường và OT
            # Giờ thường tối đa 8 tiếng/ngày
            regular_hours = min(work_time, 8)
            overtime_hours = max(0, work_time - 8)
            
            result_data.append({
                'employee_name': employee,
                'date': date,
                'check_in_original': check_in,
                'check_out_original': check_out,
                'check_in_adjusted': adjusted_check_in,
                'total_time_with_lunch': total_hours,
                'work_hours': work_time,
                'regular_hours': regular_hours,
                'overtime_hours': overtime_hours,
                'number_of_punches': len(times)
            })
        else:
            # Không đủ dữ liệu chấm công
            result_data.append({
                'employee_name': employee,
                'date': date,
                'check_in_original': times[0] if times else None,
                'check_out_original': None,
                'check_in_adjusted': None,
                'total_time_with_lunch': 0,
                'work_hours': 0,
                'regular_hours': 0,
                'overtime_hours': 0,
                'number_of_punches': len(times)
            })
    
    return pd.DataFrame(result_data)

# Áp dụng tiền xử lý nếu dữ liệu đã được đọc
if 'df' in globals():
    df_clean_v2 = preprocess_timesheet_data_v2(df)
    print("Dữ liệu sau khi tiền xử lý:")
    print(df_clean_v2[['employee_name', 'date', 'time_stamp']].head(10))
    print(f"\nSố bản ghi: {len(df_clean_v2)}")
    print(f"Số nhân viên: {df_clean_v2['employee_name'].nunique()}")
    print(f"Khoảng thời gian: {df_clean_v2['date'].min()} đến {df_clean_v2['date'].max()}")
else:
    print("Chưa có dữ liệu để xử lý. Hãy chạy cell đọc file trước.")

Dữ liệu sau khi tiền xử lý:
    employee_name       date time_stamp
114   LÊ KIM NGỌC 2025-05-21   07:30:00
115   LÊ KIM NGỌC 2025-05-21   18:21:28
116   LÊ KIM NGỌC 2025-05-22   07:30:00
117   LÊ KIM NGỌC 2025-05-22   17:34:12
118   LÊ KIM NGỌC 2025-05-23   07:30:00
119   LÊ KIM NGỌC 2025-05-23   22:01:57
120   LÊ KIM NGỌC 2025-05-24   07:30:00
121   LÊ KIM NGỌC 2025-05-24   21:38:35
122   LÊ KIM NGỌC 2025-05-25   07:30:00
123   LÊ KIM NGỌC 2025-05-25   17:38:59

Số bản ghi: 212
Số nhân viên: 4
Khoảng thời gian: 2025-05-21 00:00:00 đến 2025-06-20 00:00:00


## 4. Define Work Hour Calculation Rules

Định nghĩa các quy tắc tính giờ công:
- Giờ vào sớm nhất: 7h30
- Nghỉ trưa: 1 tiếng

In [15]:
def parse_time(time_str):
    """
    Chuyển đổi string thời gian thành datetime object
    """
    if pd.isna(time_str):
        return None
    
    # Nếu đã là datetime object
    if isinstance(time_str, datetime):
        return time_str.time()
    
    # Nếu là string
    try:
        # Thử các format khác nhau
        for fmt in ['%H:%M:%S', '%H:%M', '%I:%M:%S %p', '%I:%M %p']:
            try:
                return datetime.strptime(str(time_str), fmt).time()
            except ValueError:
                continue
        
        # Nếu không match format nào, thử extract giờ:phút
        import re
        match = re.search(r'(\d{1,2}):(\d{2})', str(time_str))
        if match:
            hour, minute = int(match.group(1)), int(match.group(2))
            return time(hour, minute)
            
    except Exception as e:
        print(f"Lỗi parse time {time_str}: {e}")
    
    return None

def apply_work_rules(check_in_time, check_out_time):
    """
    Áp dụng quy tắc tính giờ công:
    - Nếu vào trước 7h30 thì tính là 7h30
    - Trừ 1 tiếng nghỉ trưa
    """
    if not check_in_time or not check_out_time:
        return 0
    
    # Giờ vào sớm nhất là 7h30
    earliest_start = time(7, 30)
    
    # Nếu vào trước 7h30 thì tính là 7h30
    if check_in_time < earliest_start:
        adjusted_check_in = earliest_start
    else:
        adjusted_check_in = check_in_time
    
    # Tính tổng thời gian làm việc
    check_in_dt = datetime.combine(datetime.today(), adjusted_check_in)
    check_out_dt = datetime.combine(datetime.today(), check_out_time)
    
    # Nếu check_out < check_in (qua ngày), thêm 1 ngày
    if check_out_dt < check_in_dt:
        check_out_dt += timedelta(days=1)
    
    total_time = check_out_dt - check_in_dt
    total_hours = total_time.total_seconds() / 3600
    
    # Trừ 1 tiếng nghỉ trưa
    work_hours = max(0, total_hours - 1)
    
    return round(work_hours, 2)

def calculate_daily_hours(df):
    """
    Tính số giờ làm việc hàng ngày cho từng nhân viên
    """
    result_data = []
    
    for _, row in df.iterrows():
        try:
            # Parse thời gian
            check_in = parse_time(row.get('check_in'))
            check_out = parse_time(row.get('check_out'))
            
            # Tính giờ công
            work_hours = apply_work_rules(check_in, check_out)
            
            result_data.append({
                'employee_name': row.get('employee_name', ''),
                'date': row.get('date', ''),
                'check_in_original': row.get('check_in'),
                'check_out_original': row.get('check_out'), 
                'check_in_adjusted': check_in,
                'check_out': check_out,
                'work_hours': work_hours
            })
            
        except Exception as e:
            print(f"Lỗi xử lý dòng {row.name}: {e}")
            result_data.append({
                'employee_name': row.get('employee_name', ''),
                'date': row.get('date', ''),
                'check_in_original': row.get('check_in'),
                'check_out_original': row.get('check_out'),
                'check_in_adjusted': None,
                'check_out': None,
                'work_hours': 0
            })
    
    return pd.DataFrame(result_data)

print("Các hàm tính toán giờ công đã được định nghĩa!")

Các hàm tính toán giờ công đã được định nghĩa!


## 5. Calculate Daily Work Hours

Tính toán số giờ làm việc hàng ngày cho từng nhân viên

In [16]:
# Tính toán giờ công với logic mới
if 'df_clean_v2' in globals():
    print("Đang tính toán giờ công và giờ OT...")
    df_work_hours = calculate_work_hours_v2(df_clean_v2)
    
    print(f"Đã tính toán xong cho {len(df_work_hours)} ngày làm việc")
    print("\nKết quả mẫu:")
    print(df_work_hours.head(10))
    
    # Hiển thị thống kê cơ bản
    print(f"\nTổng số nhân viên: {df_work_hours['employee_name'].nunique()}")
    print(f"Tổng số ngày làm việc: {len(df_work_hours)}")
    print(f"Tổng giờ làm việc: {df_work_hours['work_hours'].sum():.2f} giờ")
    print(f"Tổng giờ thường: {df_work_hours['regular_hours'].sum():.2f} giờ")
    print(f"Tổng giờ OT: {df_work_hours['overtime_hours'].sum():.2f} giờ")
    print(f"Trung bình giờ làm việc/ngày: {df_work_hours['work_hours'].mean():.2f} giờ")
    print(f"Trung bình giờ OT/ngày: {df_work_hours['overtime_hours'].mean():.2f} giờ")
    
    # Hiển thị các trường hợp thiếu dữ liệu chấm công
    incomplete_data = df_work_hours[df_work_hours['number_of_punches'] < 2]
    if len(incomplete_data) > 0:
        print(f"\nCảnh báo: {len(incomplete_data)} ngày có dữ liệu chấm công không đầy đủ:")
        print(incomplete_data[['employee_name', 'date', 'number_of_punches']].head())
    
else:
    print("Chưa có dữ liệu để tính toán. Hãy chạy cell tiền xử lý trước đó.")

Đang tính toán giờ công và giờ OT...
Đã tính toán xong cho 101 ngày làm việc

Kết quả mẫu:
  employee_name       date check_in_original check_out_original  \
0   LÊ KIM NGỌC 2025-05-21          07:30:00           18:21:28   
1   LÊ KIM NGỌC 2025-05-22          07:30:00           17:34:12   
2   LÊ KIM NGỌC 2025-05-23          07:30:00           22:01:57   
3   LÊ KIM NGỌC 2025-05-24          07:30:00           21:38:35   
4   LÊ KIM NGỌC 2025-05-25          07:30:00           17:38:59   
5   LÊ KIM NGỌC 2025-05-26          07:30:00           21:27:14   
6   LÊ KIM NGỌC 2025-05-27          07:30:00           21:44:01   
7   LÊ KIM NGỌC 2025-05-28          07:30:00           21:26:31   
8   LÊ KIM NGỌC 2025-05-29          07:30:00           18:37:48   
9   LÊ KIM NGỌC 2025-05-30          07:30:00           21:29:24   

  check_in_adjusted  total_time_with_lunch  work_hours  regular_hours  \
0          07:30:00              10.857778    9.857778            8.0   
1          07:30:00      

## 6. Group Data by Employee and Date

Nhóm dữ liệu theo tên nhân viên và ngày

In [17]:
# Nhóm và thống kê dữ liệu theo nhân viên
if 'df_work_hours' in globals():
    # Dữ liệu đã được group theo nhân viên và ngày trong hàm calculate_work_hours_v2
    df_final = df_work_hours.copy()
    df_final['date'] = pd.to_datetime(df_final['date'])
    
    # Sắp xếp theo tên nhân viên và ngày
    df_final = df_final.sort_values(['employee_name', 'date'])
    
    print("=== DỮ LIỆU CHI TIẾT THEO NHÂN VIÊN VÀ NGÀY ===")
    print(df_final[['employee_name', 'date', 'work_hours', 'regular_hours', 'overtime_hours']].head(15))
    
    # Thống kê tổng hợp theo nhân viên
    print("\n=== THỐNG KÊ TỔNG HỢP THEO NHÂN VIÊN ===")
    employee_summary_v2 = df_final.groupby('employee_name').agg({
        'work_hours': ['sum', 'mean', 'count'],
        'regular_hours': 'sum',
        'overtime_hours': 'sum'
    }).round(2)
    
    employee_summary_v2.columns = ['Tổng giờ LV', 'TB giờ/ngày', 'Số ngày', 'Tổng giờ thường', 'Tổng giờ OT']
    print(employee_summary_v2)
    
    # Thống kê theo ngày
    print("\n=== THỐNG KÊ THEO NGÀY ===")
    daily_summary_v2 = df_final.groupby('date').agg({
        'work_hours': ['sum', 'mean'],
        'regular_hours': 'sum',
        'overtime_hours': 'sum',
        'employee_name': 'count'
    }).round(2)
    
    daily_summary_v2.columns = ['Tổng giờ LV', 'TB giờ/người', 'Tổng giờ thường', 'Tổng giờ OT', 'Số người']
    print(daily_summary_v2.head(10))
    
    # Top nhân viên có nhiều giờ OT
    print("\n=== TOP NHÂN VIÊN CÓ NHIỀU GIỜ OT ===")
    ot_ranking = df_final.groupby('employee_name')['overtime_hours'].sum().sort_values(ascending=False)
    top_ot = ot_ranking[ot_ranking > 0].head(10)
    for i, (name, hours) in enumerate(top_ot.items(), 1):
        print(f"{i:2d}. {name}: {hours:.2f} giờ OT")
    
    if len(top_ot) == 0:
        print("Không có nhân viên nào làm OT trong khoảng thời gian này.")
    
else:
    print("Chưa có dữ liệu giờ công để nhóm. Hãy chạy cell tính toán trước đó.")

=== DỮ LIỆU CHI TIẾT THEO NHÂN VIÊN VÀ NGÀY ===
   employee_name       date  work_hours  regular_hours  overtime_hours
0    LÊ KIM NGỌC 2025-05-21    9.857778            8.0        1.857778
1    LÊ KIM NGỌC 2025-05-22    9.070000            8.0        1.070000
2    LÊ KIM NGỌC 2025-05-23   13.532500            8.0        5.532500
3    LÊ KIM NGỌC 2025-05-24   13.143056            8.0        5.143056
4    LÊ KIM NGỌC 2025-05-25    9.149722            8.0        1.149722
5    LÊ KIM NGỌC 2025-05-26   12.953889            8.0        4.953889
6    LÊ KIM NGỌC 2025-05-27   13.233611            8.0        5.233611
7    LÊ KIM NGỌC 2025-05-28   12.941944            8.0        4.941944
8    LÊ KIM NGỌC 2025-05-29   10.130000            8.0        2.130000
9    LÊ KIM NGỌC 2025-05-30   12.990000            8.0        4.990000
10   LÊ KIM NGỌC 2025-05-31   13.596111            8.0        5.596111
11   LÊ KIM NGỌC 2025-06-02   13.133056            8.0        5.133056
12   LÊ KIM NGỌC 2025-06-03  

## 7. Generate Summary Report

Tạo báo cáo tổng hợp và xuất kết quả

In [18]:
# Tạo báo cáo tổng hợp và xuất file với giờ thường và OT
if 'df_final' in globals():
    # Tạo báo cáo chi tiết
    print("=== BÁO CÁO CHI TIẾT GIỜ CÔNG & OT ===")
    print(f"Thời gian tạo báo cáo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Tổng số bản ghi: {len(df_final)}")
    print(f"Số nhân viên: {df_final['employee_name'].nunique()}")
    print(f"Khoảng thời gian: {df_final['date'].min().strftime('%Y-%m-%d')} đến {df_final['date'].max().strftime('%Y-%m-%d')}")
    print(f"Tổng giờ làm việc: {df_final['work_hours'].sum():.2f} giờ")
    print(f"Tổng giờ thường: {df_final['regular_hours'].sum():.2f} giờ")
    print(f"Tổng giờ OT: {df_final['overtime_hours'].sum():.2f} giờ")
    
    # Xuất file Excel
    try:
        # Tạo file Excel với nhiều sheet
        output_file_v2 = 'timesheet_report_v2.xlsx'
        
        with pd.ExcelWriter(output_file_v2, engine='openpyxl') as writer:
            # Sheet 1: Dữ liệu chi tiết
            df_final.to_excel(writer, sheet_name='Chi tiết giờ công & OT', index=False)
            
            # Sheet 2: Thống kê theo nhân viên
            employee_stats_v2 = df_final.groupby('employee_name').agg({
                'work_hours': ['sum', 'mean', 'count'],
                'regular_hours': ['sum', 'mean'],
                'overtime_hours': ['sum', 'mean']
            }).round(2)
            employee_stats_v2.columns = [
                'Tổng giờ LV', 'TB giờ LV/ngày', 'Số ngày làm',
                'Tổng giờ thường', 'TB giờ thường/ngày',
                'Tổng giờ OT', 'TB giờ OT/ngày'
            ]
            employee_stats_v2.to_excel(writer, sheet_name='Thống kê nhân viên')
            
            # Sheet 3: Thống kê theo ngày
            daily_stats_v2 = df_final.groupby('date').agg({
                'work_hours': ['sum', 'mean'],
                'regular_hours': 'sum',
                'overtime_hours': 'sum',
                'employee_name': 'count'
            }).round(2)
            daily_stats_v2.columns = ['Tổng giờ LV', 'TB giờ/người', 'Tổng giờ thường', 'Tổng giờ OT', 'Số người']
            daily_stats_v2.to_excel(writer, sheet_name='Thống kê theo ngày')
            
            # Sheet 4: Ranking OT
            ot_ranking_df = df_final.groupby('employee_name').agg({
                'overtime_hours': ['sum', 'mean', 'count']
            }).round(2)
            ot_ranking_df.columns = ['Tổng giờ OT', 'TB giờ OT/ngày', 'Số ngày có OT']
            ot_ranking_df = ot_ranking_df.sort_values('Tổng giờ OT', ascending=False)
            ot_ranking_df.to_excel(writer, sheet_name='Ranking OT')
            
            # Sheet 5: Dữ liệu gốc
            if 'df_clean_v2' in globals():
                df_clean_v2.to_excel(writer, sheet_name='Dữ liệu gốc', index=False)
        
        print(f"\nĐã xuất báo cáo ra file: {output_file_v2}")
        
    except Exception as e:
        print(f"Lỗi khi xuất file Excel: {e}")
        
        # Xuất CSV backup
        csv_file_v2 = 'timesheet_report_v2.csv'
        df_final.to_csv(csv_file_v2, index=False, encoding='utf-8-sig')
        print(f"Đã xuất backup ra file CSV: {csv_file_v2}")
    
    # Hiển thị top nhân viên
    print("\n=== TOP 10 NHÂN VIÊN NHIỀU GIỜ LÀM VIỆC NHẤT ===")
    top_workers = df_final.groupby('employee_name')['work_hours'].sum().sort_values(ascending=False).head(10)
    for i, (name, hours) in enumerate(top_workers.items(), 1):
        regular = df_final[df_final['employee_name'] == name]['regular_hours'].sum()
        overtime = df_final[df_final['employee_name'] == name]['overtime_hours'].sum()
        print(f"{i:2d}. {name}: {hours:.2f}h (Thường: {regular:.2f}h, OT: {overtime:.2f}h)")
    
    # Hiển thị top ngày OT nhiều nhất
    print("\n=== TOP 10 NGÀY CÓ NHIỀU GIỜ OT NHẤT ===")
    top_ot_days = df_final.groupby('date')['overtime_hours'].sum().sort_values(ascending=False).head(10)
    for i, (date, hours) in enumerate(top_ot_days.items(), 1):
        if hours > 0:
            print(f"{i:2d}. {date.strftime('%Y-%m-%d')}: {hours:.2f} giờ OT")
        
else:
    print("Chưa có dữ liệu để tạo báo cáo. Hãy chạy tất cả các cell trước đó.")

=== BÁO CÁO CHI TIẾT GIỜ CÔNG & OT ===
Thời gian tạo báo cáo: 2025-08-07 17:21:40
Tổng số bản ghi: 101
Số nhân viên: 4
Khoảng thời gian: 2025-05-21 đến 2025-06-20
Tổng giờ làm việc: 1010.12 giờ
Tổng giờ thường: 807.97 giờ
Tổng giờ OT: 202.15 giờ

Đã xuất báo cáo ra file: timesheet_report_v2.xlsx

=== TOP 10 NHÂN VIÊN NHIỀU GIỜ LÀM VIỆC NHẤT ===
 1. LÊ KIM NGỌC: 289.59h (Thường: 208.00h, OT: 81.59h)
 2. TRẦN THỊ NGỌC DUYÊN: 246.30h (Thường: 216.00h, OT: 30.30h)
 3. NGUYỄN THỊ ÚT NHI: 239.78h (Thường: 200.00h, OT: 39.78h)
 4. TRẦN THU THẢO: 234.45h (Thường: 183.97h, OT: 50.48h)

=== TOP 10 NGÀY CÓ NHIỀU GIỜ OT NHẤT ===
 1. 2025-06-06: 16.44 giờ OT
 2. 2025-06-02: 12.72 giờ OT
 3. 2025-05-22: 10.87 giờ OT
 4. 2025-05-23: 10.78 giờ OT
 5. 2025-06-16: 10.53 giờ OT
 6. 2025-06-07: 10.46 giờ OT
 7. 2025-06-05: 10.43 giờ OT
 8. 2025-06-09: 10.14 giờ OT
 9. 2025-05-27: 10.03 giờ OT
10. 2025-05-28: 9.94 giờ OT


## 8. Tạo Dữ Liệu Mẫu (Nếu Không Có File)

Nếu file Excel không tồn tại, bạn có thể chạy cell dưới để tạo dữ liệu mẫu và test hệ thống